In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format',lambda x : '%.6f' % x)
pd.set_option("max_rows", 100)

import os
import gc
import warnings
warnings.filterwarnings('ignore')


DATA_PATH = '../data/'
CLEAN_PATH = '../clean/'
CACHE_PATH = '../cache/'
DATA_FILES = ['2019-Oct.csv', '2019-Nov.csv', '2019-Dec.csv',]
cleaned_target = CLEAN_PATH + f"processed_data.h5"

## 数据解释

数据集来源：[eCommerce Events History in Cosmetics Shop | Kaggle](https://www.kaggle.com/mkechinov/ecommerce-events-history-in-cosmetics-shop?select=2020-Feb.csv)

数据集内容：包含来自某中型化妆品在线商店网站 2019 年 10 月 - 2020 年 2 月的行为数据。

数据集文件：
- 2019-Dec.csv
- 2019-Nov.csv
- 2019-Oct.csv
- 2020-Feb.csv
- 2020-Jan.csv

注：本次分析仅涉及 2019 第四季度的数据，即 2019-Dec.csv，2019-Nov.csv，2019-Oct.csv。

字段解释：

|字段|数据类型|解释|
|-|-|-|
|event_type|字符串|行为类型：{view(浏览)，cart(加入购物车)，purchase(购买)，remove_from_cart(移出购物车)}|
|event_time|datetime|行为发生时间|
|user_id|非负整数|用户 id|
|product_id|非负整数|商品 id|
|category_id|非负整数|分类 id|
|brand|字符串|品牌名称|
|price|正实数|商品价格|

## 数据读取

读取所有数据文件并合并：

In [2]:
data = pd.DataFrame()
for f in DATA_FILES:
    df = pd.read_csv(f"{DATA_PATH}{f}")
    data = data.append(df, ignore_index=True)
    print(f"{f}: {df.shape}")
gc.collect()
data.info()

2019-Oct.csv: (4102283, 9)
2019-Nov.csv: (4635837, 9)
2019-Dec.csv: (3533286, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12271406 entries, 0 to 12271405
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 842.6+ MB


查看首尾几条数据：

In [3]:
data.head(3).append(data.tail(3))

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,NaN,runail,2.620000,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,NaN,runail,2.620000,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,NaN,lovely,13.480000,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9
12271403,2019-12-31 23:59:51 UTC,view,59975,1487580012096782476,NaN,NaN,7.140000,595414541,4c6d80bb-5dd3-4fbb-b592-187b51db2753
12271404,2019-12-31 23:59:52 UTC,view,5775982,1783999063314661546,NaN,NaN,11.900000,397780878,7e8a2b85-153a-44eb-a71f-b748fde14fcc
12271405,2019-12-31 23:59:57 UTC,view,5635090,1487580005754995573,NaN,NaN,4.440000,420652863,546f6af3-a517-4752-a98b-80c4c5860711


## 数据预处理

### 数据转换

In [4]:
data['event_time'] = pd.to_datetime(data['event_time'].apply(lambda x: x[:-4]))
data.dtypes

event_time       datetime64[ns]
event_type               object
product_id                int64
category_id               int64
category_code            object
brand                    object
price                   float64
user_id                   int64
user_session             object
dtype: object

### 重复值

查看完全重复的记录所占比例：

In [5]:
data.duplicated().sum() / data.shape[0]

0.05245592884792501

由于记录中的行为时间精确到秒，一般情况下，用户对物品在某一秒只会有一种行为，所以理论上，用户对物品在某个时刻的行为不应该存在重复记录。查看这部分重复记录所占的比例：

In [6]:
data[['product_id', 'user_id', 'event_time',]].duplicated().sum() / data.shape[0]

0.053170842852074164

鉴于上述原因且该部分数据占整体比例仅约 5.3%，故只保留用户对物品在某个时刻的第一条行为记录，其余重复记录视为异常数据予以删除：

In [7]:
data = data.drop_duplicates(subset=['product_id', 'user_id', 'event_time',], keep='first')
data[['product_id', 'user_id', 'event_time',]].duplicated().any()

False

### 缺失值

查看各特征缺失率：

In [8]:
data.isnull().sum() / data.shape[0]

event_time      0.000000
event_type      0.000000
product_id      0.000000
category_id     0.000000
category_code   0.983170
brand           0.419638
price           0.000000
user_id         0.000000
user_session    0.000176
dtype: float64

可见存在缺失值特征有：category_code，brand，user_session。

#### category_code

假设：category_id 与 category_code 一一对应。

查看 category_id 可能对应的 category_code 数量：

In [9]:
cc_null = data['category_code'].isnull()
icc = data[~cc_null][['category_id', 'category_code']].drop_duplicates().groupby('category_id').size()
cic = data[~cc_null][['category_id', 'category_code']].drop_duplicates().groupby('category_code').size()
icc.nunique(), icc.nunique()

(1, 1)

经验证，非缺失部分 category_id 与 category_code 一一对应。

基于以下原因，决定对 category_code 特征予以删除：
1. 缺失值比例高达 98%
2. 非缺失部分与 category_id 一一对应，隐含信息可由 category_id 代替

In [10]:
data = data.drop(columns=['category_code'])
data.columns

Index(['event_time', 'event_type', 'product_id', 'category_id', 'brand',
       'price', 'user_id', 'user_session'],
      dtype='object')

#### user_session

查看用户 session 个数统计情况，发现大部分集中在，但存在少量用户 session 个数原高于均值：

In [11]:
usc = data[['user_id', 'user_session']].drop_duplicates(keep='first').groupby('user_id').size()
usc.describe()

count   1012561.000000
mean          2.616639
std          17.811744
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max       13260.000000
dtype: float64

In [12]:
pd.DataFrame(usc.value_counts(normalize=True).sort_index(ascending=True).cumsum(), columns=['cumsum_percent'])

,cumsum_percent
1,0.677271
2,0.826802
3,0.883299
4,0.912804
5,0.930860
...,...
1823,0.999996
2048,0.999997
3049,0.999998
5135,0.999999


统计非空 user_session 字符串长度，都为 36：

In [13]:
data[~data['user_session'].isnull()]['user_session'].apply(lambda x: len(x)).unique()

array([36], dtype=int64)

将非空 session 以 “-”为分隔符进行拆分，尝试寻找规律：

In [14]:
session_split = data[~data['user_session'].isnull()]['user_session'].str.split('-', expand=True)
session_split.columns = [f"session_{i}" for i in range(session_split.shape[1])]
session_split

,session_0,session_1,session_2,session_3,session_4
0,26dd6e6e,4dac,4778,8d2c,92e149dab885
1,26dd6e6e,4dac,4778,8d2c,92e149dab885
2,49e8d843,adf3,428b,a2c3,fe8bc6a307c9
3,26dd6e6e,4dac,4778,8d2c,92e149dab885
4,49e8d843,adf3,428b,a2c3,fe8bc6a307c9
...,...,...,...,...,...
12271401,e4a2d47c,a956,4c46,8176,745f52ea664b
12271402,2dde9867,9e71,4a64,880d,aa68b66aae6d
12271403,4c6d80bb,5dd3,4fbb,b592,187b51db2753
12271404,7e8a2b85,153a,44eb,a71f,b748fde14fcc


session 拆分后各段的唯一值，数据量较大且无明显统计规律：

In [15]:
session_split.nunique()

session_0    2647753
session_1      65536
session_2      42450
session_3      47140
session_4    2648576
dtype: int64

由于数据源中缺乏对 session 字段的详细描述，也无法通过拆分 session 各段得到明确有效的信息，故决定予以删除：

In [16]:
data.drop(columns=['user_session'], inplace=True)
data.columns

Index(['event_time', 'event_type', 'product_id', 'category_id', 'brand',
       'price', 'user_id'],
      dtype='object')

#### brand

查看每个 product_id 可能对应的 brand 数量，发现一个 product_id 可能对应 1 或 2 个 brand：

In [17]:
pbc = data[['product_id', 'brand',]].drop_duplicates().groupby('product_id').size()
pbc.unique()

array([1, 2], dtype=int64)

查看每个 product_id 可能对应的 category_id 数量，发现也存在一个 product_id 对应 1 或 2 个  category_id 的情况：

In [18]:
pcc = data[['product_id', 'category_id',]].drop_duplicates().groupby('product_id').size()
pcc.unique()

array([1, 2], dtype=int64)

查看 brand 与 category_id 的对应情况，发现 product_id 与 category_id 大多不唯一对应：

In [19]:
bcb = data[['brand', 'category_id',]].drop_duplicates().groupby('brand').size()
bcb.unique()

array([  5,  12,   2,   9,  15,   7,   1,  19,   6,   3,   4,  21,  24,
        14,  27,  34,  17,   8,  11,  18,  13,  52,  22,  40,  25,  20,
        16,  10,  54, 139,  26,  30,  35,  28,  56,  29,  31, 112,  23],
      dtype=int64)

In [20]:
bcc = data[['brand', 'category_id',]].drop_duplicates().groupby('category_id').size()
bcc.unique()

array([ 2,  3,  1, 30, 10,  5,  8,  6, 19, 11,  4, 13, 14, 16, 12,  9,  7,
       17, 56, 51, 32, 31, 18, 33, 26, 41, 28, 15, 23, 22, 54, 39, 46, 24,
       36, 47, 20, 44, 34, 21], dtype=int64)

假设通过 product_id 和 category_id 能唯一确定 brand，然而经验证，一组 product_id 和 category_id 可能对应 1 或 2 个 brand：

In [21]:
pcbc = data[['product_id', 'category_id', 'brand',]].drop_duplicates().groupby(['product_id', 'category_id',]).size()
pcbc.unique()

array([1, 2], dtype=int64)

假设通过 product_id 和 brand 能唯一确定 category_id，然而经验证，一组 product_id 和 brand 可能对应 1 或 2 个 category_id：

In [22]:
pcbc = data[['product_id', 'category_id', 'brand',]].drop_duplicates().groupby(['product_id', 'brand',]).size()
pcbc.unique()

array([1, 2], dtype=int64)

因此，初步断定 product_id 的 category_id 和 brand 在此期间可能发生变化，下面通过观察 product_id 在不同时间下的 category_id 和 brand 信息进行验证。

首先查看 product_id 对应（'category_id', 'brand'）的组数分布：

In [23]:
pcbpc = data[['product_id', 'category_id', 'brand',]].drop_duplicates().groupby(['product_id',], as_index=False).size()
pcbpc['size'].value_counts()

1    44898
2     4418
3       25
Name: size, dtype: int64

以 'MISSING' 填充 brand 缺失值，获取每组（'product_id', 'category_id', 'brand'）的日期区间：

In [24]:
data['brand'].fillna('MISSING', inplace=True)
data['date'] = data['event_time'].dt.date
pcbe = data.groupby(by=['product_id', 'category_id', 'brand',])[['date']].agg([np.min, np.max]).reset_index()
pcbe.columns = ['product_id', 'category_id', 'brand', 'first_date', 'last_date']
pcbe

,product_id,category_id,brand,first_date,last_date
0,3752,1487580005411062629,cnd,2019-10-01,2019-12-27
1,3762,1487580005411062629,cnd,2019-10-01,2019-12-31
2,3763,1487580005411062629,cnd,2019-10-01,2019-12-27
3,3771,1487580005411062629,MISSING,2019-10-08,2019-12-30
4,3774,1487580005411062629,cnd,2019-10-01,2019-12-30
...,...,...,...,...,...
53804,5917175,1487580012096782476,MISSING,2019-12-30,2019-12-30
53805,5917175,1487580012096782476,herbolive,2019-12-30,2019-12-31
53806,5917177,1487580011627020412,MISSING,2019-12-30,2019-12-30
53807,5917177,1487580011627020412,herbolive,2019-12-30,2019-12-30


观察对应 2 组（'category_id', 'brand'）的 product_id 的日期区间重叠情况，发现存在日期区间的重叠：

In [25]:
pcbe2 = pd.merge(left=pcbe, right=pcbpc[pcbpc['size'] == 2][['product_id']]
        ,how='inner', on=['product_id'])
# pcbe2
pcbe2_asc = pcbe2.copy()
pcbe2_asc['r'] = pcbe2_asc.groupby('product_id')['first_date'].rank(ascending=False)
pcbe2_asc = pcbe2_asc[pcbe2_asc['r'] == 2]
# pcbe2_asc
pcbe2_desc = pcbe2.copy()
pcbe2_desc['r'] = pcbe2_desc.groupby('product_id')['first_date'].rank(ascending=True)
pcbe2_desc = pcbe2_desc[pcbe2_desc['r'] == 2]
# pcbe2_desc
pcbe_delta = pd.merge(left=pcbe2_asc, right=pcbe2_desc, how='inner', 
         on=['product_id'], suffixes=['_l', '_r'])
pcbe_delta['delta'] = pcbe_delta['first_date_r'] - pcbe_delta['last_date_l']
pcbe_delta.drop(columns=['r_l', 'r_r',], inplace=True)
pcbe_delta['delta'].describe()

count                         3564
mean     4 days 12:57:46.666666666
std      8 days 20:17:50.274073724
min             -82 days +00:00:00
25%                1 days 00:00:00
50%                2 days 00:00:00
75%                6 days 00:00:00
max               62 days 00:00:00
Name: delta, dtype: object

1. 观察区间无重叠的情况：

   现象：这部分数据仅是 category_id 或 brand 其中一项发生变更。
   
   推测：产生这部分数据的原因可能为信息调整或数据错误。
   
   措施：由于这部分数据的不确定性，且占比较底，予以删除。

In [26]:
pcbe_delta[pcbe_delta['delta'].dt.days >= 0]

,product_id,category_id_l,brand_l,first_date_l,last_date_l,category_id_r,brand_r,first_date_r,last_date_r,delta
0,4382,1487580006174425994,cnd,2019-10-01,2019-11-30,2196456817758831535,cnd,2019-12-01,2019-12-31,1 days
1,4497,1487580006895846315,runail,2019-10-01,2019-11-30,2195085258272014535,runail,2019-12-01,2019-12-31,1 days
2,5524,1487580006199591819,ingarden,2019-10-22,2019-11-30,2196456817641391019,ingarden,2019-12-01,2019-12-31,1 days
3,5525,1487580006199591819,ingarden,2019-10-01,2019-11-30,2196456817641391019,ingarden,2019-12-01,2019-12-29,1 days
4,5526,1487580006199591819,ingarden,2019-10-01,2019-11-30,2196456817641391019,ingarden,2019-12-01,2019-12-31,1 days
...,...,...,...,...,...,...,...,...,...,...
3559,5916988,1783999073758478650,MISSING,2019-12-29,2019-12-29,1783999073758478650,beautific,2019-12-30,2019-12-31,1 days
3560,5917157,1783999072332415142,MISSING,2019-12-30,2019-12-30,1783999072332415142,herbolive,2019-12-31,2019-12-31,1 days
3561,5917158,1783999072332415142,MISSING,2019-12-30,2019-12-30,1783999072332415142,herbolive,2019-12-31,2019-12-31,1 days
3562,5917162,1487580012096782476,MISSING,2019-12-30,2019-12-30,1487580012096782476,herbolive,2019-12-31,2019-12-31,1 days


2. 观察重叠区间情况

    1. 初步发现这部分数据的 category_id 相等，但 brand 都存在缺失值 
    2. 经验证，这部分数据 category_id 相同，但一部分 brand 缺失，剩余部分 brand 唯一
    
  推测：这部分数据仅为 brand 缺失。

  措施：使用未缺失部分的 brand 对缺失部分进行补全。

In [27]:
# 1. 初步发现这部分数据的 category_id 相等，但 brand 都存在缺失值 
pcbe_delta0 = pcbe_delta[pcbe_delta['delta'].dt.days < 0]
pcbe_delta0

,product_id,category_id_l,brand_l,first_date_l,last_date_l,category_id_r,brand_r,first_date_r,last_date_r,delta
232,5657300,1487580011517968503,bodyton,2019-10-01,2019-12-31,1487580011517968503,MISSING,2019-11-13,2019-11-13,-48 days
233,5657301,1487580011517968503,bodyton,2019-10-01,2019-12-31,1487580011517968503,MISSING,2019-11-12,2019-11-13,-49 days
234,5657720,1487580012457492629,bodyton,2019-10-02,2019-12-31,1487580012457492629,MISSING,2019-11-13,2019-11-13,-48 days
235,5657778,1487580012457492629,bodyton,2019-10-01,2019-12-31,1487580012457492629,MISSING,2019-11-12,2019-11-12,-49 days
236,5657781,1487580009143992338,bodyton,2019-10-01,2019-12-31,1487580009143992338,MISSING,2019-11-12,2019-11-13,-49 days
237,5657919,1487580010662330448,bodyton,2019-10-01,2019-12-30,1487580010662330448,MISSING,2019-11-12,2019-11-13,-48 days
309,5678040,1487580010662330448,bodyton,2019-10-02,2019-12-30,1487580010662330448,MISSING,2019-11-13,2019-11-13,-47 days
310,5678043,1487580010662330448,bodyton,2019-10-01,2019-12-29,1487580010662330448,MISSING,2019-11-12,2019-11-13,-47 days
311,5678044,1487580010662330448,bodyton,2019-10-01,2019-12-28,1487580010662330448,MISSING,2019-11-12,2019-11-12,-46 days
312,5678046,1487580010662330448,bodyton,2019-10-01,2019-12-29,1487580010662330448,MISSING,2019-11-12,2019-11-13,-47 days


In [28]:
# 2. 经验证，这部分数据 category_id 相同，但一部分 brand 缺失，剩余部分 brand 唯一：
all((pcbe_delta0['brand_r'] == 'MISSING').astype(int) + (pcbe_delta0['brand_l'] == 'MISSING').astype(int)) == 1

True

3. 观察对应 3 组（'category_id', 'brand'）的 product_id 的日期区间重叠情况：
    
   现象：**同一个 product_id 的 category_id 和 brand 分别在不同的时间区间先后发生变更**，不存在时间区间重叠的情况。
    
   推测：现象产生的原因可能是信息更新存在延迟，或是日志系统定期校对产生变更。

   措施：由于这部分数据的不确定性，且占比极底，予以删除。

In [29]:
# 1. 不存在时间区间重叠的情况
pcbe3 = pd.merge(left=pcbe, right=pcbpc[pcbpc['size'] == 3][['product_id']]
        ,how='inner', on=['product_id'])
pcbe3 = pcbe3.sort_values(by=['product_id', 'first_date'])
pcbe3['next_fd'] = pcbe3.groupby('product_id')['first_date'].shift(-1)
pcbe3['diff'] = pcbe3['next_fd'] - pcbe3['last_date']
pcbe3

,product_id,category_id,brand,first_date,last_date,next_fd,diff
0,37042,1487580006895846315,candy,2019-10-01,2019-11-30,2019-12-01,1 days
1,37042,1783999064136745198,candy,2019-12-01,2019-12-04,2019-12-04,0 days
2,37042,1783999064136745198,runail,2019-12-04,2019-12-30,NaN,NaT
3,37072,1487580006895846315,candy,2019-10-01,2019-11-30,2019-12-01,1 days
4,37072,1783999064136745198,candy,2019-12-01,2019-12-02,2019-12-04,2 days
5,37072,1783999064136745198,runail,2019-12-04,2019-12-30,NaN,NaT
8,5683272,1487580011408916594,candy,2019-10-03,2019-11-28,2019-12-03,5 days
6,5683272,1487580011383750769,candy,2019-12-03,2019-12-03,2019-12-05,2 days
7,5683272,1487580011383750769,swarovski,2019-12-05,2019-12-30,NaN,NaT
11,5861706,1819693959081886239,fly,2019-10-01,2019-11-29,2019-12-01,2 days


In [30]:
# 2. 同一个 product_id 的不同 category_id 和 brand 所处区间的时间间隔统计信息
pcbe3['diff'].describe()

count                           50
mean               2 days 07:40:48
std      3 days 10:46:32.462745387
min                0 days 00:00:00
25%                0 days 00:00:00
50%                1 days 00:00:00
75%                2 days 00:00:00
max               15 days 00:00:00
Name: diff, dtype: object

综上，针对 brand 处理措施如下：

1. 对应 2 组（'category_id', 'brand'）的 product_id，使用已知的 brand 填充缺失值
2. 剩余包含 2 组以上（'category_id', 'brand'）的 product_id，对相关记录进行删除

至此，**清理后的数据中每个 prodcut_id 对应的 category_id 和 brand 唯一**。

In [31]:
# 1.
pcbe_delta0_mapper = dict()
for _, row in pcbe_delta0.iterrows():
    pcbe_delta0_mapper[row['product_id']] = row['brand_r'] if row['brand_l'] == 'MISSING' else row['brand_l']
t = data['product_id'].map(pcbe_delta0_mapper)
data.loc[~t.isnull(), 'brand'] = data['product_id'].map(pcbe_delta0_mapper)
# 2.
pcbpc = data[['product_id', 'category_id', 'brand',]].drop_duplicates().groupby(['product_id',], as_index=False).size()
# pcbpc['size'].value_counts()
del data['date']
ori_len = data.shape[0]
data = pd.merge(left=data, right=pcbpc[pcbpc['size'] == 1][['product_id']], how='inner', on=['product_id'])
cur_len = data.shape[0]
print(f"Delete {(ori_len - cur_len) / ori_len:.2%} data.")

Delete 6.98% data.


### 异常值

查看各特征统计信息，发现价格存在负数：

In [32]:
data.describe()

,product_id,category_id,price,user_id
count,10807841.000000,10807841.000000,10807841.000000,10807841.000000
mean,5485136.685628,1548591980670611968.000000,8.765987,509907610.163238
std,1292268.550271,161393803347272128.000000,19.745965,84527327.689798
min,3752.000000,1487580004807082752.000000,-79.370000,1120748.000000
25%,5723529.000000,1487580005754995456.000000,2.060000,474660576.000000
50%,5809855.000000,1487580008246412288.000000,4.110000,545903799.000000
75%,5853035.000000,1487580013388628224.000000,7.140000,565791530.000000
max,5917178.000000,2235524499636224256.000000,317.460000,595414541.000000


In [33]:
data.describe(include=object)

,event_type,brand
count,10807841,10807841
unique,4,245
top,view,MISSING
freq,5256601,4644687


查看价格为负数的记录：

In [34]:
data[data['price'] < 0]

,event_time,event_type,product_id,category_id,brand,price,user_id
8878444,2019-10-01 19:10:56,purchase,5716857,1487580014042939619,MISSING,-23.810000,552507528
8878445,2019-10-13 16:46:01,purchase,5716857,1487580014042939619,MISSING,-23.810000,559820267
8878446,2019-10-16 11:41:06,purchase,5716857,1487580014042939619,MISSING,-23.810000,461943726
8878447,2019-10-28 07:41:54,purchase,5716857,1487580014042939619,MISSING,-23.810000,385939606
8878448,2019-10-29 14:34:24,purchase,5716857,1487580014042939619,MISSING,-23.810000,554081558
8878449,2019-11-04 11:54:29,purchase,5716857,1487580014042939619,MISSING,-23.810000,516725846
8878450,2019-11-10 20:16:38,purchase,5716857,1487580014042939619,MISSING,-23.810000,549736485
8878451,2019-11-18 07:06:17,purchase,5716857,1487580014042939619,MISSING,-23.810000,565309368
8878452,2019-11-22 15:18:19,purchase,5716857,1487580014042939619,MISSING,-23.810000,541269785
8878453,2019-11-29 14:00:23,purchase,5716857,1487580014042939619,MISSING,-23.810000,480927132


可见，价格为负的记录对应行为都为‘购买’且同属于一个分类。

查看改目录下对应商品及价格：

In [35]:
data[data['category_id'] == 1487580014042939619][['product_id', 'price']].value_counts()

product_id  price     
5670256     15.870000     398
5670257     -15.870000     17
5716859     -47.620000     13
5716857     -23.810000     12
5716855     -7.940000       7
5716861     -79.370000      4
dtype: int64

发现该分类仅有一项商品价格正常，其余商品价格皆为负数。是数据有误还是业务问题需要需进一步探究。

由于价格为负的记录占比极小，故将此部分数据视为异常值予以删除：

In [36]:
data = data[data['price'] >= 0].reset_index(drop=True)
data

,event_time,event_type,product_id,category_id,brand,price,user_id
0,2019-10-01 00:00:00,cart,5773203,1487580005134238553,runail,2.620000,463240011
1,2019-10-01 00:12:50,purchase,5773203,1487580005134238553,runail,2.620000,463240011
2,2019-10-01 04:46:10,cart,5773203,1487580005134238553,runail,2.620000,520672512
3,2019-10-01 04:58:19,purchase,5773203,1487580005134238553,runail,2.620000,520672512
4,2019-10-01 05:28:01,view,5773203,1487580005134238553,runail,2.620000,434540110
...,...,...,...,...,...,...,...
10807783,2019-12-31 09:30:10,view,5916331,1487580007675986893,MISSING,0.000000,383988490
10807784,2019-12-31 10:16:35,view,5913480,1542195323827388674,dermacol,13.970000,595222024
10807785,2019-12-31 12:42:36,view,5916024,1487580008112194531,MISSING,0.000000,595267919
10807786,2019-12-31 17:14:54,view,5691801,1487580005427839846,irisk,3.440000,487975953


### 唯一值

查看各特征的唯一值个数：

In [37]:
data.nunique()

event_time     5078720
event_type           4
product_id       44920
category_id        478
brand              245
price             2646
user_id         969369
dtype: int64

经验证，不存在仅含唯一值的特征。

## 保存预处理数据

将预处理后的数据保存到 hdf 文件：

In [38]:
group_key = 'q4'
data.to_hdf(cleaned_target, key=group_key)
print(f"清洗后的数据保存路径：{cleaned_target}，key：{group_key}")

清洗后的数据保存路径：../clean/processed_data.h5，key：q4


In [39]:
data = pd.read_hdf(cleaned_target, key='q4')
data

,event_time,event_type,product_id,category_id,brand,price,user_id
0,2019-10-01 00:00:00,cart,5773203,1487580005134238553,runail,2.620000,463240011
1,2019-10-01 00:12:50,purchase,5773203,1487580005134238553,runail,2.620000,463240011
2,2019-10-01 04:46:10,cart,5773203,1487580005134238553,runail,2.620000,520672512
3,2019-10-01 04:58:19,purchase,5773203,1487580005134238553,runail,2.620000,520672512
4,2019-10-01 05:28:01,view,5773203,1487580005134238553,runail,2.620000,434540110
...,...,...,...,...,...,...,...
10807783,2019-12-31 09:30:10,view,5916331,1487580007675986893,MISSING,0.000000,383988490
10807784,2019-12-31 10:16:35,view,5913480,1542195323827388674,dermacol,13.970000,595222024
10807785,2019-12-31 12:42:36,view,5916024,1487580008112194531,MISSING,0.000000,595267919
10807786,2019-12-31 17:14:54,view,5691801,1487580005427839846,irisk,3.440000,487975953
